In [11]:
import pandas as pd

heur = pd.read_csv('results/optimizer.csv', sep='\t', header=None)
heur.columns = ['key', 'k', 'm',
                'heuristic_name', 'time',
                'status', 'used', '%fixed', 'solution']

heur.loc[:, 'pois'] = heur['key'].str.split(';').str[0].str.split(' ').str[0].astype(int)
heur.loc[:, 'sensors'] = heur['key'].str.split(';').str[0].str.split(' ').str[1].astype(int)
heur.loc[:, 'seed'] = heur['key'].str.split(';').str[-1].astype(int)
heur.loc[:, 'time'] = heur['time'] / 1_000_000
heur.loc[:, 'iterations'] = heur['heuristic_name'].str.split('_').str[-1].apply(
    lambda h: int(h) if h.isdigit() else 0
)
heur.loc[:, 'heuristic_name'] = heur['heuristic_name'].apply(
    lambda h: '_'.join(h.split('_')[:-1])
).replace({'': 'dinic'})

preprocessing = heur.drop(columns=['status', 'solution', 'key']
                         ).copy(deep=True)

# print(sorted(preprocessing.columns))

grp_cols = ['heuristic_name', 'pois', 'sensors', 'k', 'm']
group_prep = preprocessing[
    grp_cols+['%fixed',
              'iterations',
              'time']
].sort_values(grp_cols).groupby(grp_cols).mean().reset_index(drop=False)
            
group_prep[group_prep['heuristic_name'] == 'best_reuse'].drop(columns=grp_cols)

,%fixed,iterations,time
0,0.918000,0.0,0.086508
1,0.857000,4.7,0.056838
2,0.847000,0.0,0.069613
3,0.803000,9.2,0.056815
4,0.802000,3.8,0.094388
5,0.783000,0.0,0.100410
6,0.971000,0.0,0.270952
7,0.950001,5.0,0.357249
8,0.943666,0.0,0.320317
9,0.930998,9.6,0.281249


In [12]:
raw = pd.read_parquet('/data/results/parsed.parquet')

print(len(raw))

print(raw['gurobi_model_name'].value_counts())

sorted(raw.columns)

6420
dinic_gurobi_multi_flow          360
max_reuse_gurobi_single_flow     360
gurobi_y_binary_single_flow      360
gurobi_single_flow               360
max_reuse_gurobi_multi_flow      360
best_reuse_gurobi_single_flow    359
best_reuse_gurobi_multi_flow     359
min_reuse_gurobi_single_flow     359
no_reuse_gurobi_multi_flow       359
no_reuse_gurobi_single_flow      359
min_reuse_gurobi_multi_flow      359
dinic_gurobi_single_flow         356
max_flood_gurobi_multi_flow      354
gurobi_multi_flow                354
min_flood_gurobi_single_flow     353
max_flood_gurobi_single_flow     353
min_flood_gurobi_multi_flow      351
gurobi_y_binary_multi_flow       345
Name: gurobi_model_name, dtype: int64


['area_side',
 'communication_radius',
 'coverage_radius',
 'gurobi_binary_variables_count',
 'gurobi_bound',
 'gurobi_bound_c',
 'gurobi_columns_count',
 'gurobi_continuous_variables_count',
 'gurobi_heuristic_objective_value',
 'gurobi_initial_binary_variables_count',
 'gurobi_initial_columns_count',
 'gurobi_initial_continuous_variables_count',
 'gurobi_initial_integer_variables_count',
 'gurobi_initial_non_zero_count',
 'gurobi_initial_rows_count',
 'gurobi_integer_variables_count',
 'gurobi_logs',
 'gurobi_mip_gap',
 'gurobi_model_name',
 'gurobi_node_count',
 'gurobi_non_zero_count',
 'gurobi_objective_value',
 'gurobi_optimal',
 'gurobi_presolve_removed_columns',
 'gurobi_presolve_removed_rows',
 'gurobi_presolve_time',
 'gurobi_rows_count',
 'gurobi_run_time',
 'gurobi_setup_time',
 'gurobi_simplex_iterations_count',
 'gurobi_solution',
 'gurobi_solutions_count',
 'gurobi_variable_x_size',
 'gurobi_variable_y_size',
 'heuristic_name',
 'heuristic_objective_value',
 'heuristic_s

In [13]:
ls /data

done.json                   instances.50.csv       random_instances.tar.gz
gupta_instance_classes.csv  instances.csv          results/
gurobi_dcc.lic              instances.tar.gz       small_instance_classes.csv
gurobi_isa.lic              kcmc_gurobi.lic        small_instances.csv
gurobi_mm8.lic              old.tar.gz
instances.10.csv            preprocessing.parquet


In [14]:
with open('done.json', 'w') as fout:
    import json
    json.dump(
        sorted(raw[['instance_key', 'k', 'm', 'gurobi_model_name']].drop_duplicates().to_records(index=None).tolist()),
        fout
    )

In [15]:

# print(len(raw), sorted(raw.columns))

raw = raw[[
    'gurobi_model_name', 'y_binary',
    'pois', 'sensors', 'k', 'm', 'random_seed',
    'gurobi_optimal', 'gurobi_objective_value',
    # 'gurobi_initial_rows_count', 'gurobi_initial_columns_count', 'gurobi_initial_non_zero_count',
    'gurobi_rows_count', 'gurobi_columns_count', 'gurobi_non_zero_count',
    'gurobi_node_count',
    'gurobi_bound_c',         # LB
    'gurobi_objective_value', # UB
    'gurobi_mip_gap',
    #'gurobi_setup_time', 'gurobi_presolve_time', 'gurobi_run_time', 'total_time',
    'gurobi_run_time',
    'gurobi_logs'
]]
raw.columns = [
    'model', 'y_binary',
    'pois', 'sensors', 'k', 'm', 'seed', 'opt', 'sol', 'rows', 'columns', 'non_zero',
    'nodes',
    'lb',
    'ub',
    'gap',
    #'setup_time', 'presolve_time', 'run_time', 'total_time',
    'time',
    'log'
]


multi_flow = raw[
    (raw['model'] == 'gurobi_multi_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)

single_flow = raw[
    (raw['model'] == 'gurobi_single_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)


dinic_multi_flow = raw[
    (raw['model'] == 'dinic_gurobi_multi_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)

dinic_single_flow = raw[
    (raw['model'] == 'dinic_gurobi_single_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)


min_flood_multi_flow = raw[
    (raw['model'] == 'min_flood_gurobi_multi_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)

min_flood_single_flow = raw[
    (raw['model'] == 'min_flood_gurobi_single_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)


max_flood_multi_flow = raw[
    (raw['model'] == 'max_flood_gurobi_multi_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)

max_flood_single_flow = raw[
    (raw['model'] == 'max_flood_gurobi_single_flow')
  & (raw['y_binary'] == False)
].drop(columns=['y_binary']).copy(deep=True)



reuse_results = []
for prefix in ['no_reuse', 'min_reuse', 'max_reuse', 'best_reuse']:
    for suffix in ['_gurobi_single_flow', '_gurobi_multi_flow']:
        reuse_results.append(raw[(raw['model'] == prefix+suffix)
                               & (raw['y_binary'] == False)
                             ].drop(columns=['y_binary']).copy(deep=True))

[len(i) for i in [
    single_flow,       multi_flow,
    dinic_single_flow, dinic_multi_flow,
    min_flood_single_flow, min_flood_multi_flow,
        max_flood_single_flow, max_flood_multi_flow]
  + reuse_results]

[360,
 354,
 356,
 360,
 353,
 351,
 353,
 354,
 359,
 359,
 359,
 359,
 360,
 360,
 359,
 359]

In [16]:
group_cols = ['pois', 'sensors', 'k', 'm']

result = {}
for df in [
    single_flow,               multi_flow,
    dinic_single_flow,         dinic_multi_flow,
    min_flood_single_flow,     min_flood_multi_flow,
        max_flood_single_flow,     max_flood_multi_flow
] + reuse_results:

# for df in [reuse_results[-1]]:
    
    model = df.iloc[0]['model']
    result[model] = []
    for group, sdf in df.sort_values(group_cols).groupby(group_cols):
        
        seeds = set(sdf['seed'].unique())
        
        prep = preprocessing[preprocessing['seed'].isin(seeds)].copy()
        prep = prep[prep['heuristic_name'].apply(lambda n: model.startswith(n))]
        # print(model, len(prep))
        
        res = dict(zip(['|P|', '|I|', 'K', 'M'], list(group)))
        sensors = float(group[1])

        optimal = sdf[sdf['opt']]
        # print(group, 'NOT OPTIMAL SEEDS:', sorted(sdf[~sdf['opt']]['seed'].tolist()))
        
        integer = sdf[  (sdf['lb'] < 1_000_000)  & (sdf['lb'] > 0)
                      & (sdf['ub'] < 1_000_000)  & (sdf['ub'] > 0)
                      & (sdf['gap'] < 1_000_000) & (sdf['gap'] >= 0)]
        
        has_lb = integer
        has_ub = integer
        has_gap = integer
        
        not_optimal = sdf[~sdf['opt']]
        
        res.update({
            # OPT: Quantos casos de status OPTIMAL
            '#opt': len(optimal),
            # INT: Quantos casos de valor de solucao nao-infinita
            '#int': len(integer),
            # LINES: quantas linhas no modelo SEM INCLUIR PRESOLVE
            '#lines': round(sdf['rows'].mean(), 2),
            # COLUMNS: quantas colunas no modelo SEM INCLUIR PRESOLVE
            '#columns': round(sdf['columns'].mean(), 2),
            # NON-ZERO: quantas variaveis nao-zero no modelo SEM INCLUIR PRESOLVE
            '#non-zero': round(sdf['non_zero'].mean(), 2),
            # NODES: quantos nodos explorados no SIMPLEX, nao incluindo o nodo-raiz
            '#nodes': round(sdf['nodes'].mean(), 2),
            
            # LB: media do LOWER-BOUND
            'LB': has_lb['lb'].mean(),
            # NODES: media do UPPER-BOUND
            'UB': has_ub['ub'].mean(),
            # GAP: media do GAP
            '%GAP': has_gap['gap'].mean(),
            
            # time: media do tempo de processamento do SIMPLEX, sem incluir PRESOLVE
            # APENAS DAS INSTANCIAS EM QUE ENCONTRAMOS OS OTIMOS
            'time(s)': round(optimal['time'].mean(), 2),

            # FIXED: quantidade de sensores desativados em preprocessamento
            '%fixed': -1 if (len(prep) == 0) else round(prep['%fixed'].mean(), 2),
            # Iterations: quantidade de iteracoes de preprocessamento
            #'Iterations': -1 if (len(prep) == 0) else round(prep['iterations'].mean(), 2),
            # pre-time: tempo de preprocessamento
            'pre-time(s)': -1 if (len(prep) == 0) else round(prep['time'].mean(), 2)
        })
        result[model].append(res)
        
    result[model] = (pd.DataFrame(result[model])
                     .apply(lambda col: col.astype(str).str.lower().str.strip())
                     .replace({'nan': '*', '-1': ' ', '-1.0': ' '}))


(100, 100, 1, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 1) NOT OPTIMAL SEEDS: []
(100, 100, 3, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 3) NOT OPTIMAL SEEDS: []
(100, 300, 1, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 2) NOT OPTIMAL SEEDS: []
(100, 300, 3, 1) NOT OPTIMAL SEEDS: []
(100, 300, 3, 2) NOT OPTIMAL SEEDS: []
(100, 300, 3, 3) NOT OPTIMAL SEEDS: []
(100, 500, 1, 1) NOT OPTIMAL SEEDS: [192953821, 193136057, 193210805, 193218673, 193325622, 193403161, 193455793]
(100, 500, 2, 1) NOT OPTIMAL SEEDS: [226380584, 226474569, 226522771, 226604827, 226690050, 226904765]
(100, 500, 2, 2) NOT OPTIMAL SEEDS: [231851231, 231874945, 231908212, 231992085, 232077505, 232150338, 232178372, 232233041, 232305496]
(100, 500, 3, 1) NOT OPTIMAL SEEDS: [228880070, 228938840, 228986049, 228986154, 229060678, 229156830, 229221446]
(100, 500, 3, 2) NOT OPTIMAL SEEDS: [158015982, 158040146, 158046

(100, 100, 1, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 1) NOT OPTIMAL SEEDS: []
(100, 100, 3, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 3) NOT OPTIMAL SEEDS: []
(100, 300, 1, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 2) NOT OPTIMAL SEEDS: []
(100, 300, 3, 1) NOT OPTIMAL SEEDS: []
(100, 300, 3, 2) NOT OPTIMAL SEEDS: []
(100, 300, 3, 3) NOT OPTIMAL SEEDS: []
(100, 500, 1, 1) NOT OPTIMAL SEEDS: []
(100, 500, 2, 1) NOT OPTIMAL SEEDS: []
(100, 500, 2, 2) NOT OPTIMAL SEEDS: []
(100, 500, 3, 1) NOT OPTIMAL SEEDS: [228784260, 228911470, 229221446]
(100, 500, 3, 2) NOT OPTIMAL SEEDS: [158159851]
(100, 500, 3, 3) NOT OPTIMAL SEEDS: []
(200, 100, 1, 1) NOT OPTIMAL SEEDS: []
(200, 100, 2, 1) NOT OPTIMAL SEEDS: []
(200, 100, 2, 2) NOT OPTIMAL SEEDS: []
(200, 100, 3, 1) NOT OPTIMAL SEEDS: []
(200, 100, 3, 2) NOT OPTIMAL SEEDS: []
(200, 100, 3, 3) NOT OPTIMAL SEEDS: []
(200, 300, 1, 1) NOT OPT

(200, 100, 1, 1) NOT OPTIMAL SEEDS: []
(200, 100, 2, 1) NOT OPTIMAL SEEDS: []
(200, 100, 2, 2) NOT OPTIMAL SEEDS: []
(200, 100, 3, 1) NOT OPTIMAL SEEDS: []
(200, 100, 3, 2) NOT OPTIMAL SEEDS: []
(200, 100, 3, 3) NOT OPTIMAL SEEDS: []
(200, 300, 1, 1) NOT OPTIMAL SEEDS: []
(200, 300, 2, 1) NOT OPTIMAL SEEDS: []
(200, 300, 2, 2) NOT OPTIMAL SEEDS: []
(200, 300, 3, 1) NOT OPTIMAL SEEDS: []
(200, 300, 3, 2) NOT OPTIMAL SEEDS: []
(200, 300, 3, 3) NOT OPTIMAL SEEDS: []
(200, 500, 1, 1) NOT OPTIMAL SEEDS: []
(200, 500, 2, 1) NOT OPTIMAL SEEDS: []
(200, 500, 2, 2) NOT OPTIMAL SEEDS: []
(200, 500, 3, 1) NOT OPTIMAL SEEDS: []
(200, 500, 3, 2) NOT OPTIMAL SEEDS: []
(200, 500, 3, 3) NOT OPTIMAL SEEDS: []
(100, 100, 1, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 1) NOT OPTIMAL SEEDS: []
(100, 100, 3, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 3) NOT OPTIMAL SEEDS: []
(100, 300, 1, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 1) NOT OPTI

(200, 100, 3, 1) NOT OPTIMAL SEEDS: []
(200, 100, 3, 2) NOT OPTIMAL SEEDS: []
(200, 100, 3, 3) NOT OPTIMAL SEEDS: []
(200, 300, 1, 1) NOT OPTIMAL SEEDS: []
(200, 300, 2, 1) NOT OPTIMAL SEEDS: []
(200, 300, 2, 2) NOT OPTIMAL SEEDS: []
(200, 300, 3, 1) NOT OPTIMAL SEEDS: []
(200, 300, 3, 2) NOT OPTIMAL SEEDS: []
(200, 300, 3, 3) NOT OPTIMAL SEEDS: []
(200, 500, 1, 1) NOT OPTIMAL SEEDS: []
(200, 500, 2, 1) NOT OPTIMAL SEEDS: []
(200, 500, 2, 2) NOT OPTIMAL SEEDS: []
(200, 500, 3, 1) NOT OPTIMAL SEEDS: []
(200, 500, 3, 2) NOT OPTIMAL SEEDS: []
(200, 500, 3, 3) NOT OPTIMAL SEEDS: []
(100, 100, 1, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 1) NOT OPTIMAL SEEDS: []
(100, 100, 2, 2) NOT OPTIMAL SEEDS: [176429384]
(100, 100, 3, 1) NOT OPTIMAL SEEDS: []
(100, 100, 3, 2) NOT OPTIMAL SEEDS: []
(100, 100, 3, 3) NOT OPTIMAL SEEDS: []
(100, 300, 1, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 1) NOT OPTIMAL SEEDS: []
(100, 300, 2, 2) NOT OPTIMAL SEEDS: []
(100, 300, 3, 1) NOT OPTIMAL SEEDS: []
(100, 300, 3, 2)

In [26]:
result['no_reuse_gurobi_multi_flow'][[
    # '|P|', '|I|', 'K', 'M',
    '#opt', '#int', 
    # '#lines', '#columns', '#non-zero', 
    '#nodes', 
    'LB', 'UB', '%GAP', 
    'time(s)'
]]

#.drop(
#    columns=['#lines', '#columns', '#non-zero']
#          + ['|P|', '|I|', 'K', 'M']
#)

,#opt,#int,#nodes,LB,UB,%GAP,time(s)
0,10,10,0.0,6.7,6.7,0.0,0.03
1,10,10,0.2,12.3,12.3,0.0,0.05
2,10,10,0.3,12.2,12.2,0.0,7.55
3,10,10,0.0,17.8,17.8,0.0,0.1
4,10,10,0.0,17.8,17.8,0.0,0.4
5,10,10,0.5,19.0,19.0,0.0,1.44
6,10,10,0.0,6.3,6.3,0.0,0.03
7,10,10,0.2,11.2,11.2,0.0,0.06
8,10,10,0.4,12.9,12.9,0.0,0.33
9,10,10,0.0,17.0,17.0,0.0,0.12


## INSPECT

In [20]:
len(multi_flow[multi_flow['lb'] == 0.0])/len(multi_flow), \
len(single_flow[single_flow['lb'] == 0.0])/len(single_flow)

(0.3050847457627119, 0.21666666666666667)

In [21]:
def print_lb_0(name, df):
    print(name, '\t', round(len(df[df['lb'] == 0.0])/len(df), 3),
                '\t', round(len(df[df['ub'] == 0.0])/len(df), 3))
    
    
print_lb_0('multi_flow\t', multi_flow)
print_lb_0('single_flow\t', single_flow)
print_lb_0('dinic_multi_flow', dinic_multi_flow)
print_lb_0('dinic_single_flow', dinic_single_flow)
print_lb_0('min_flood_multi_flow', min_flood_multi_flow)
print_lb_0('min_flood_single_flow', min_flood_single_flow)
print_lb_0('max_flood_multi_flow', max_flood_multi_flow)
print_lb_0('max_flood_single_flow', max_flood_single_flow)

multi_flow	 	 0.305 	 0.0
single_flow	 	 0.217 	 0.0
dinic_multi_flow 	 0.117 	 0.0
dinic_single_flow 	 0.0 	 0.0
min_flood_multi_flow 	 0.131 	 0.0
min_flood_single_flow 	 0.0 	 0.0
max_flood_multi_flow 	 0.169 	 0.0
max_flood_single_flow 	 0.023 	 0.0


In [53]:
df = reuse_results[-1]

# sorted(df['seed'].unique())

In [54]:
interest = \
df[(df['pois'] == 100)
 & (df['sensors'] == 300)
 & (df['k'] == 3)
 & (df['m'] == 3)
]

interest = df[df['seed'] == 176429384]

print(len(interest))

linha = interest.iloc[0]

print(linha[['lb', 'ub']], '\n', '='*40)

print(linha['log'])

1
lb    1000000000000000015902891109759918046836080856...
ub                                                  NaN
Name: 3410, dtype: object 
Gurobi 9.1.2 (linux64) logging started Wed Sep 21 00:24:07 2022
Changed value of parameter LogFile to /tmp/gurobi.log
   Prev:   Default: 
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter LogToConsole to 0
   Prev: 1  Min: 0  Max: 1  Default: 1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 2 physical cores, 4 logical processors, using up to 1 threads
Optimize a model with 6515 rows, 84030 columns and 123162 nonzeros
Model fingerprint: 0x7e205ea6
Variable types: 84000 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 4975 rows and 79677 columns
Presolve time: 0.14s
Presolved: 1540 rows, 4353 columns, 13612 nonzeros
V

In [38]:
linha.keys()

Index(['model', 'pois', 'sensors', 'k', 'm', 'seed', 'opt', 'sol', 'rows',
       'columns', 'non_zero', 'nodes', 'lb', 'ub', 'gap', 'time', 'log'],
      dtype='object')

In [ ]:
linha = \
df[(df['pois'] == 100)
          & (df['sensors'] == 300)
          & (df['k'] == 3)
          & (df['m'] == 3)
          & (df['ub'] < 1_000_000)
          & (df['ub'] > 0)
          & (df['lb'] > 0.0)
].iloc[0]

print(linha[['lb', 'ub']], '\n', '='*40)

print(linha['log'])

In [ ]:
print(multi_flow[multi_flow['lb'] == 0.0].iloc[0]['log'])

In [ ]:
multi_flow.loc[1025]

In [ ]:
print(multi_flow.loc[1025]['log'])
# Temos hsol
# nao temos ub, lb

In [ ]:
print(multi_flow.loc[1110]['log'])
# Temos best bound (lb), solution
# nao temos ub, hsol